# Market Rates API Example

## Setup
Install the Signal Ocean SDK:
```
pip install signal-ocean
```
Set your subscription key acquired here: https://apis.signalocean.com/profile

In [5]:
signal_ocean_api_key = '' #replace with your subscription key

## Market Rates API
The Market Rates API retrieves available routes and market rates. First create connection towards
Market Rates API in order to find available routes by vessel class:

In [6]:
from signal_ocean import Connection
from signal_ocean.market_rates import MarketRatesAPI
from datetime import datetime

connection = Connection(api_key=signal_ocean_api_key)
mr_api = MarketRatesAPI(connection)

#### Get the available vessel classes
Retrieve the vessel classes for which we can get routes or market rates:

In [7]:
vessel_classes = mr_api.get_vessel_classes()
print(vessel_classes)

(VesselClass(id=60, vessel_type_id=6, from_size=70001, to_size=250000, name='VLGC', vessel_type='LPG', defining_size='CubicSize', size='cbm'), VesselClass(id=61, vessel_type_id=6, from_size=28001, to_size=70000, name='Midsize/LGC', vessel_type='LPG', defining_size='CubicSize', size='cbm'), VesselClass(id=62, vessel_type_id=6, from_size=10001, to_size=28000, name='Handy', vessel_type='LPG', defining_size='CubicSize', size='cbm'), VesselClass(id=63, vessel_type_id=6, from_size=0, to_size=10000, name='Small', vessel_type='LPG', defining_size='CubicSize', size='cbm'), VesselClass(id=69, vessel_type_id=3, from_size=220000, to_size=550000, name='VLOC', vessel_type='Dry', defining_size='DeadWeight', size='kt'), VesselClass(id=70, vessel_type_id=3, from_size=120000, to_size=219999, name='Capesize', vessel_type='Dry', defining_size='DeadWeight', size='kt'), VesselClass(id=72, vessel_type_id=3, from_size=85000, to_size=119999, name='Post Panamax Dry', vessel_type='Dry', defining_size='DeadWeight

#### Get routes for a vessel class
Based on the vessel classes that we got above we can request available routes for Aframax vessels

In [4]:
afra_vessel_class = next(vc for vc in vessel_classes if vc.name=='Aframax')
afra_routes = mr_api.get_routes(afra_vessel_class.id)
print(afra_routes)

HTTPError: 401 Client Error: Access Denied for url: https://signaloceantestapims.azure-api.net/market-rates/api/routes/86

#### Get the market rates
Now we can request the market rate on a given date for one of the routes we got in previous step or
for all routes matching this vessel class

In [5]:
mr_api.get_market_rates(datetime(2020, 1, 2), route_id=afra_routes[0].id)
mr_api.get_market_rates(datetime(2020, 1, 2), vessel_class_id=afra_vessel_class.id)

(VesselType(id=1, name='Tanker'), VesselType(id=3, name='Dry'), VesselType(id=4, name='Container'), VesselType(id=5, name='LNG'), VesselType(id=6, name='LPG'))
